In [74]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv(dotenv_path="D:\College\python\coldEmailGenerator\.env")

# # Access the API key
api_key = os.getenv("GROQ_API_KEY")
api_key


'<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>'

In [ ]:

llm = ChatGroq(
    temperature=0, 
    groq_api_key=api_key, 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [77]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-33460")
page_data = loader.load().pop().page_content
print(page_data)



Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsFIND A STOREBECOME A MEMBERSIGN UP FOR EMAILSEND US FEEDBACKSTUDENT DISCOUNTSGET HELPGET HELPOrder StatusDeliveryReturnsPayment OptionsC

In [78]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [79]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

# # First add IDs to each item
# for i, item in enumerate(json_res):
#     item['id'] = str(i)

# # Then create dictionary with ID as key
# json_dict = {item['id']: item for item in json_res}

[{'role': 'Administrative Support',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Advanced Innovation',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Air Manufacturing Innovation',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Aviation', 'experience': None, 'skills': None, 'description': None},
 {'role': 'Communications',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Customer Service',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Design', 'experience': None, 'skills': None, 'description': None},
 {'role': 'Digital', 'experience': None, 'skills': None, 'description': None},
 {'role': 'Facilities',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Finance & Accounting',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Government & Public Affairs',
  'experience': None,
  'skills': None,
  'desc

In [80]:
type(json_res)

list

In [81]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [82]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [83]:
links = collection.query(query_texts=['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [84]:
job

{'role': 'Advanced Innovation',
 'experience': None,
 'skills': None,
 'description': None}

In [85]:
job = json_res[1]
job['skills']


In [86]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlocking Advanced Innovation with AtliQ's AI & Software Expertise

Dear Hiring Manager,

I came across the job description for the Advanced Innovation role and was impressed by the scope of the position. As a Business Development Executive at AtliQ, I'd like to introduce our company's capabilities in driving innovation through AI and software consulting.

At AtliQ, we empower enterprises to achieve scalability, process optimization, cost reduction, and heightened overall efficiency by integrating automated tools into their business processes. Our team of experts has a proven track record of delivering tailored solutions that cater to the unique needs of our clients.

Our portfolio showcases our expertise in machine learning and Python development, with notable projects including:
https://example.com/ml-python-portfolio and https://example.com/python-portfolio. These examples demonstrate our ability to leverage cutting-edge technologies to drive innovation and solve complex pr